# Prompt Template


Tutorial này bao gồm cách tạo và sử dụng các **prompt templates** bằng **LangChain**.

**Prompt templates** rất cần thiết để tạo ra các **prompts** động và linh hoạt, đáp ứng nhiều trường hợp sử dụng khác nhau, chẳng hạn như lịch sử hội thoại, đầu ra có cấu trúc và truy vấn chuyên biệt.

Trong tutorial này, chúng ta sẽ khám phá các phương pháp để tạo các đối tượng **PromptTemplate**, áp dụng **partial variables**, quản lý các **templates** thông qua tệp YAML và tận dụng các công cụ nâng cao như **ChatPromptTemplate** và **MessagePlaceholder** để tăng cường chức năng.


## Setup environment

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True, dotenv_path="../.env")


True

### Method 1. Using the `from_template()` method

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("what is {a} + {b}")
# prompt.invoke({"a": 1, "b": 2})
prompt.format(a=1, b=2)

'what is 1 + 2'

### Method 2. Creating a `PromptTemplate` object and a prompt all at once

In [8]:
template = "Thành phố của {country} là gì?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"]
)
prompt.format(country="Việt Nam")

'Thành phố của Việt Nam là gì?'

In [10]:
template = "{country1} và {country2} cái nào đẹp hơn?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "Viet Nam"
    }
)

prompt.format(country1="India")

'India và Viet Nam cái nào đẹp hơn?'

In [12]:
prompt.partial_variables = {"country2": "Korea"}
prompt.format(country1="India")

'India và Korea cái nào đẹp hơn?'

### Using `partial_variables`


Sử dụng `partial_variables`, bạn có thể áp dụng hàm một phần. Điều này đặc biệt hữu ích khi có các `common variables` (biến chung) cần được chia sẻ.

Các ví dụ phổ biến là `date` (ngày) hoặc `time` (thời gian).

Giả sử bạn muốn chỉ định ngày hiện tại trong prompt của mình, việc hardcoding (mã hóa cứng) ngày vào prompt hoặc truyền nó cùng với các biến đầu vào khác có thể không thực tế. Trong trường hợp này, việc sử dụng một hàm trả về ngày hiện tại để sửa đổi prompt một phần sẽ thuận tiện hơn nhiều.


In [14]:
from datetime import datetime

def get_today():
    return datetime.now().strftime("%B %d")

tenplate = "Hôm nay là {date}, liệt kê {n} việc cần làm."

prompt = PromptTemplate(
    template=tenplate,
    input_variables=['n'],
    partial_variables={
        "date": get_today
    }
)

prompt.format(n=3)

'Hôm nay là March 08, liệt kê 3 việc cần làm.'

In [15]:
prompt.format(n=5)

'Hôm nay là March 08, liệt kê 5 việc cần làm.'

### Load Prompt Templates from `YAML` Files

You can manage prompt templates in seperate `yaml` files and load using `load_prompt`.

In [30]:
from langchain_core.prompts import load_prompt

def get_today():
    return datetime.now().strftime("%B %d")

prompt = load_prompt("prompts/fruit_color.yaml", encoding='utf-8')
prompt.format(fruit1="Apple", fruit2="Orange")

'HIHI - HAHA - What is the color of Apple and Orange?'

### ChatPromptTemplate

`ChatPromptTemplate` có thể được sử dụng để bao gồm lịch sử hội thoại như một prompt.

Các message được cấu trúc dưới dạng tuples theo định dạng `(role, message)` và được tạo thành một list.

`role`

* `system`: Một message thiết lập hệ thống, thường được sử dụng cho các prompt liên quan đến cài đặt toàn cục.
* `human`: Một message nhập liệu của người dùng.
* `ai`: Một message phản hồi của AI.


In [33]:
from langchain_core.prompts import ChatPromptTemplate

# From template
prompt = ChatPromptTemplate.from_template("What is the best thing in the {country}")
prompt.format(country="Viet nam")

'Human: What is the best thing in the Viet nam'

In [36]:
# From messsages

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " Bạn là một AI toàn năng "),
        ("ai", "HAHA, đúng là như vậy"),
        ("human", "Bạn có thể trả lời tôi một câu hỏi được không?"),
        ("ai", "Vô tư điiii"),
        ("human", "{question}")
    ]
)

prompt.format_messages(question="Ai đẹp trai nhất thế giới?")

[SystemMessage(content=' Bạn là một AI toàn năng ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='HAHA, đúng là như vậy', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Bạn có thể trả lời tôi một câu hỏi được không?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Vô tư điiii', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Ai đẹp trai nhất thế giới?', additional_kwargs={}, response_metadata={})]

### `MessagePlaceholder`

LangChain cũng cung cấp `MessagePlaceholder`, cho phép kiểm soát hoàn toàn việc hiển thị các message trong quá trình định dạng.

Điều này có thể hữu ích nếu bạn không chắc chắn nên sử dụng vai trò nào trong một message prompt template hoặc nếu bạn muốn chèn một list các message trong quá trình định dạng.


Trong LangChain, `MessagePlaceHolder` là một thành phần được sử dụng để "đặt chỗ" (placeholder) cho một hoặc nhiều tin nhắn trong luồng hội thoại. Nó không chứa nội dung cụ thể mà chỉ giữ một vị trí trong danh sách các tin nhắn (messages), để sau này bạn có thể thay thế nó bằng dữ liệu thực tế, chẳng hạn như lịch sử trò chuyện, thông tin từ người dùng, hoặc phản hồi của AI.

Nói đơn giản, nó giống như một "hộp rỗng" mà bạn đặt sẵn trong cấu trúc hội thoại, và khi cần, bạn sẽ "đổ đầy" nội dung vào đó.

##### Khi nào dùng `MessagePlaceHolder`?
- Khi bạn muốn linh hoạt trong việc thêm tin nhắn vào luồng hội thoại mà không cần xác định ngay nội dung cụ thể.
- Khi bạn làm việc với các mô hình trò chuyện (chat models) và cần giữ chỗ cho lịch sử hội thoại hoặc thông tin động (dynamic data).

##### Ví dụ dễ hiểu
Hãy tưởng tượng bạn đang xây dựng một chatbot đặt pizza. Bạn muốn chatbot nhớ lịch sử trò chuyện của khách hàng (ví dụ: "Tôi muốn pizza pepperoni" hoặc "Thêm phô mai đi") để đưa ra phản hồi phù hợp. Nhưng lịch sử này thay đổi tùy theo mỗi khách hàng, nên bạn không thể cứng nhắc viết sẵn. Đây là lúc `MessagePlaceHolder` phát huy tác dụng.

##### Code ví dụ
Dưới đây là một ví dụ đơn giản bằng Python sử dụng LangChain:

```python
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Tạo một template hội thoại
prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là trợ lý đặt pizza. Hãy giúp khách hàng một cách thân thiện!"),
    MessagesPlaceholder(variable_name="chat_history"),  # Giữ chỗ cho lịch sử trò chuyện
    ("human", "Tôi muốn đặt một chiếc pizza.")
])

# Giả lập lịch sử trò chuyện
chat_history = [
    ("human", "Tôi thích pizza pepperoni."),
    ("ai", "Được thôi, bạn muốn kích thước bao nhiêu?")
]

# Khởi tạo mô hình chat
model = ChatOpenAI()

# Tạo chuỗi (chain) để xử lý
chain = prompt | model

# Gọi chuỗi với lịch sử trò chuyện
response = chain.invoke({"chat_history": chat_history})
print(response.content)
```

##### Giải thích từng phần:
1. **`ChatPromptTemplate.from_messages`**:
   - Đây là nơi bạn định nghĩa cấu trúc hội thoại.
   - Tin nhắn hệ thống ("system") cố định: "Bạn là trợ lý đặt pizza..."
   - `MessagesPlaceholder("chat_history")`: Giữ chỗ cho lịch sử trò chuyện, tên biến là `chat_history`.
   - Tin nhắn người dùng ("human"): "Tôi muốn đặt một chiếc pizza."

2. **`chat_history`**:
   - Đây là dữ liệu thực tế sẽ được "đổ" vào `MessagesPlaceholder`. Trong ví dụ, lịch sử trò chuyện là một danh sách các tin nhắn giữa người dùng và AI.

3. **`chain.invoke`**:
   - Khi chạy, LangChain sẽ thay thế `MessagesPlaceholder` bằng `chat_history` và gửi toàn bộ nội dung (system + chat_history + human) tới mô hình AI.

##### Kết quả
Kết quả có thể là:
```
Chào bạn! Bạn đã nhắc đến pizza pepperoni rồi, giờ bạn muốn kích thước bao nhiêu cho chiếc pizza này?
```

#### So sánh đơn giản
- Không có `MessagePlaceHolder`: Bạn phải viết cứng mọi tin nhắn trong template, không linh hoạt.
- Có `MessagePlaceHolder`: Bạn để trống một "khe" và thay đổi nội dung tùy theo tình huống.

#### Ứng dụng thực tế
- Chatbot: Giữ lịch sử trò chuyện giữa người dùng và AI.
- Hỏi đáp: Chèn các câu hỏi trước đó để AI hiểu ngữ cảnh.
- Tùy chỉnh: Thêm dữ liệu động từ cơ sở dữ liệu hoặc input của người dùng.

In [38]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a summarization specialist AI assistant. Your mission is to summarize conversations using key points.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "Summarize the conversation so far in {word_count} words."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [39]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "Hello! I’m Teddy. Nice to meet you."),
        ("ai", "Nice to meet you! I look forward to working with you."),
    ],
)

print(formatted_chat_prompt)

System: You are a summarization specialist AI assistant. Your mission is to summarize conversations using key points.
Human: Hello! I’m Teddy. Nice to meet you.
AI: Nice to meet you! I look forward to working with you.
Human: Summarize the conversation so far in 5 words.
